## Fetch and prepare FWF data

### 1. Fetch

Fetch PulseWaves data of a flight line from the NYU Spatial Data Repo.

In [2]:
project='sp19'
line='161348'

# fetch pulsewaves
!../scripts/fetch.sh $project $line fwf-pw

--2020-07-31 19:28:18--  https://archive.nyu.edu/bitstream/2451/60462/5/nyu_2451_60462_fwf_plswvs_FD_190511_161348.zip
Resolving archive.nyu.edu (archive.nyu.edu)... 128.122.108.142
Connecting to archive.nyu.edu (archive.nyu.edu)|128.122.108.142|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 527670310 (503M) [application/octet-stream]
Saving to: ‘tmp1596220098.26812/nyu_2451_60462_fwf_plswvs_FD_190511_161348.zip’

nyu_2451_60462_fwf_ 100%[===================>] 503.23M  11.2MB/s    in 64s     

2020-07-31 19:29:22 (7.92 MB/s) - ‘tmp1596220098.26812/nyu_2451_60462_fwf_plswvs_FD_190511_161348.zip’ saved [527670310/527670310]

Archive:  tmp1596220098.26812/nyu_2451_60462_fwf_plswvs_FD_190511_161348.zip
  inflating: tmp1596220098.26812/nyu_2451_60462_fwf_plswvs_FD_190511_161348/10552_NYU_M2_Pulse_Waves_MTA_Resolved-Scanner1-190511_161348_1-originalpoints.pls  
  inflating: tmp1596220098.26812/nyu_2451_60462_fwf_pl

Check that the PulseWaves files have been downloaded and unpacked

In [3]:
!ls ../data/fwf-pw/*

../data/fwf-pw/10552_NYU_M2_Pulse_Waves_MTA_Resolved-Scanner1-190511_161348_1-originalpoints.pls
../data/fwf-pw/10552_NYU_M2_Pulse_Waves_MTA_Resolved-Scanner1-190511_161348_1-originalpoints.wvs
../data/fwf-pw/F_150326_115601.pls
../data/fwf-pw/F_150326_115601.wvs


Display the Pulse file header

In [4]:
pls_files = !ls ../data/fwf-pw/*.pls
first_pls_file = pls_files[0]
!java -jar "../jar/umg-cli-0.2.0-SNAPSHOT-jar-with-dependencies.jar" pulseinfo -i $first_pls_file

--------------------------------------------------
PULSE HEADER
--------------------------------------------------
fileSignature                  PulseWavesPulse
globalParameter                00000000
fileSourceID                   0
guidData1                      1900857167
guidData2                      41870
guidData3                      17123
guidData4                      
systemIdentifier               RiPROCESS 1.8.5
generatingSoftware             PulseWaves DLL 0.3 r11 (150617) by rapidlasso
fileCreationDate               149
fileCreationYear               2019
versionMajor                   0
versionMinor                   3
headerSize                     352
offsetToPulseData              9338
numberOfPulses                 4808941
pulseFormat                    0
pulseAttributes                0
pulseSize                      48
pulseCompression               0
reserved                       0
numberOfVariableLengthRecords  18
numberOfAppededVariableLengthRecords 0
tScaleF

2020-07-31 19:30:06,645 [main] DEBUG umg.core.lidar.pulsewaves.PVariableLengthRecord - VLR length: 404
--------------------------------------------------
VLR #13 of 18
--------------------------------------------------
userID                         PulseWaves_Spec
recordID                       200008
reserved                       0
recordLengthAfterHeader        404
description                    PulseWaves 0.3 r11 (150617) by rapidlasso
raw data                       [404 bytes]
----- Pulse descriptor -----
size                           92
reserved                       0
opticalCenterToAnchorPoint     0
numberOfExtraWaveBytes         0
numberOfSamplings              3
sampleUnits                    1.000
compression                    0
scannerIndex                   1
description                    1 x RP, 3 x LP, 1 x HP

Sampling descriptor #0
size                           104
reserved                       0
type                           1
channel                        3
un

### 2. Convert the data to PWMsg format

The Java module below pairs each pulse with its waveforms and encodes the data in a [Protocol Buffers](https://developers.google.com/protocol-buffers) format. The encoded data (i.e. PWMsg) are segmented to small chunks (e.g. 100000 pulses) to make them more managable. The format protocol is in the `data/protobuf/pulsewaves.proto` file.

In [5]:
!mkdir -p ../data/pwmsg
!java -jar "../jar/umg-cli-0.2.0-SNAPSHOT-jar-with-dependencies.jar" pw2proto -idir ../data/fwf-pw -odir ../data/pwmsg -meta_file ../data/metadata/$project-meta.json -segment 100000 -subseq 0 5000

2020-07-31 19:30:55,191 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - Offset: [977000.000, 173000.000, 0.000, , 0.000]
2020-07-31 19:30:55,192 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - Scale: [0.001, 0.001, 0.001, , 0.000]
2020-07-31 19:30:55,192 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - Flight ID map contains 85 entries.
2020-07-31 19:30:55,196 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - File source ID: 0
2020-07-31 19:30:55,197 [pool-1-thread-1] INFO  umg.core.lidar.protobuf.cli.PW2Proto - Processing /home/vvo/code/vvogit/sunsetpark/notebooks/../data/fwf-pw/F_150326_115601.pls
Processing /home/vvo/code/vvogit/sunsetpark/notebooks/../data/fwf-pw/F_150326_115601.pls
2020-07-31 19:30:55,198 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - File source ID: 1
2020-07-31 19:30:55,198 [pool-1-thread-2] INFO  umg.core.lidar.protobuf.cli.PW2Proto - Processing /home/vvo/code/vvogit/sunsetpark/notebooks/../data/fwf-pw/10552_NYU_M2_Pulse_Waves_MTA_Resolved-Sca

Check the PWMsg data 

In [6]:
!ls ../data/pwmsg

10552_NYU_M2_Pulse_Waves_MTA_Resolved-Scanner1-190511_161348_1-originalpoints-0.pwmsg
F_150326_115601-0.pwmsg


Check notebook [01_pwmsg.ipynb](01_pwmsg.ipynb) for how to work with the PWMsg files using Python.